# Atividade Prática: Avaliação Comparativa de Classificadores K-NN, LVQ, AD, SVM

## Importando bibliotecas e dataset pré-processado

In [1]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from scipy.stats import randint
import random

plt.rcParams['figure.figsize'] = [16, 10]

In [2]:
dataset = pd.read_csv('C:/Users/55819/Downloads/wine-processed-normalized.csv')

In [3]:
dataset = dataset.drop(columns=['Unnamed: 0'])
dataset.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,color
0,0.142473,2.188833,-2.192833,-0.744778,0.569958,-1.100140,-1.446359,1.034993,1.813090,0.193097,-0.915464,5,red
1,0.451036,3.282235,-2.192833,-0.597640,1.197975,-0.311320,-0.862469,0.701486,-0.115073,0.999579,-0.580068,5,red
2,0.451036,2.553300,-1.917553,-0.660699,1.026697,-0.874763,-1.092486,0.768188,0.258120,0.797958,-0.580068,5,red
3,3.073817,-0.362438,1.661085,-0.744778,0.541412,-0.762074,-0.986324,1.101694,-0.363868,0.327510,-0.580068,6,red
4,0.142473,2.188833,-2.192833,-0.744778,0.569958,-1.100140,-1.446359,1.034993,1.813090,0.193097,-0.915464,5,red


In [4]:
dataset_preproc = pd.get_dummies(dataset, columns=['color'])
dataset_preproc.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,color_red,color_white
0,0.142473,2.188833,-2.192833,-0.744778,0.569958,-1.100140,-1.446359,1.034993,1.813090,0.193097,-0.915464,5,1,0
1,0.451036,3.282235,-2.192833,-0.597640,1.197975,-0.311320,-0.862469,0.701486,-0.115073,0.999579,-0.580068,5,1,0
2,0.451036,2.553300,-1.917553,-0.660699,1.026697,-0.874763,-1.092486,0.768188,0.258120,0.797958,-0.580068,5,1,0
3,3.073817,-0.362438,1.661085,-0.744778,0.541412,-0.762074,-0.986324,1.101694,-0.363868,0.327510,-0.580068,6,1,0
4,0.142473,2.188833,-2.192833,-0.744778,0.569958,-1.100140,-1.446359,1.034993,1.813090,0.193097,-0.915464,5,1,0


## Dividindo entre atributos e variável-alvo

In [5]:
X = dataset_preproc.drop(columns=['quality'])
Y = dataset_preproc['quality']

## Inicializando modelos

In [6]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin

class LVQ(BaseEstimator, ClassifierMixin):
    def __init__(self, n_codebooks=10, lrate=0.3, epochs=50):
        self.n_codebooks = n_codebooks
        self.lrate = lrate
        self.epochs = epochs

    def euclidean_distance(self, row1, row2):
        return np.sqrt(np.sum((row1 - row2)**2))

    def get_best_matching_unit(self, codebooks, test_row):
        distances = [self.euclidean_distance(codebook[:-1], test_row) for codebook in codebooks]
        return codebooks[np.argmin(distances)]

    def train_codebooks(self, X, y):
        n_classes = len(np.unique(y))
        codebooks = np.array([np.append(X[np.random.choice(len(X))], y[np.random.choice(len(y))]) for _ in range(self.n_codebooks)])
        for epoch in range(self.epochs):
            rate = self.lrate * (1.0 - (epoch / float(self.epochs)))
            for row, label in zip(X, y):
                bmu = self.get_best_matching_unit(codebooks, row)
                for i in range(len(row)):
                    error = row[i] - bmu[i]
                    if bmu[-1] == label:
                        bmu[i] += rate * error
                    else:
                        bmu[i] -= rate * error
        return codebooks

    def fit(self, X, y):
        self.classes_ = np.unique(y)
        self.codebooks_ = self.train_codebooks(X, y)
        return self

    def predict(self, X):
        predictions = [self.get_best_matching_unit(self.codebooks_, row)[-1] for row in X]
        return np.array(predictions)

    def predict_proba(self, X):
        proba = []
        for x in X:
            distances = []
            classes = []
            # Calcula a distância para cada codebook
            for codebook in self.codebooks_:
                dist = self.euclidean_distance(codebook[:-1], x)
                distances.append(dist)
                classes.append(codebook[-1])
            distances = np.array(distances)
            classes = np.array(classes)
            unique_classes = self.classes_
            # Para cada classe, encontra a menor distância
            class_scores = []
            for cls in unique_classes:
                cls_distances = distances[classes == cls]
                if len(cls_distances) == 0:
                    min_dist = np.inf  # Se não houver codebooks dessa classe
                else:
                    min_dist = np.min(cls_distances)
                class_scores.append((cls, min_dist))
            # Converte distâncias em probabilidades inversas
            total = sum(1.0 / (score[1] + 1e-10) for score in class_scores)  # Evita divisão por zero
            probs = []
            for cls, dist in class_scores:
                prob = (1.0 / (dist + 1e-10)) / total
                probs.append(prob)
            proba.append(probs)
        return np.array(proba)


In [7]:
models = {}

models['DTR'] = DecisionTreeClassifier()
models['KNN'] = KNeighborsClassifier()
models['SVC'] = SVC(probability=True)
models['LVQ'] = LVQ()
models['MLP'] = MLPClassifier()

## Definição de dicionário para busca randomizada

In [8]:
params = {}

### KNN

Para a definição do dicionário que foi utilizado na função RandomizedSearchCV foram levados em considerações as informações da função KNeighborsClassifier e um exemplo de hyperparameter tuning encontrado no kaggle

Material de base: https://www.kaggle.com/code/arunimsamudra/k-nn-with-hyperparameter-tuning

*  **n_neighbors**: representa a quantidade de vizinhos que serão considerados e para isso foi utilizada a função randint da biblioteca scipy.stats para selecionar números inteiros entre 3 e 10 no RandomizedSearchCV;
*  **weights**: o parâmetro weights representa a função de peso que será utilizado na predição. Para a busca randomizada foi selecionada as duas opções da biblioteca: 'uniform' e 'distance';
* **algorithm**: este parâmetro indica qual atributo será usado pelo KNeighborsClassifier para computar os vizinhos, e, de forma similar ao parâmetro weights, foram selecionadas as quatro funções da biblioteca;
* **leaf_size**: este parâmetro não foi alterado, já que só é usado quando o algoritmo utilizado é o BallTree ou o KDTree. Além disso, como a biblioteca informa, este parâmetro afeta apenas a velocidade de construção do modelo;
* **p**: este parâmetro não foi alterado já que o parâmetro metric foi modificado;
* **metric**: para o parâmetro metric, que indica qual métrica de distância será utilizada pelo K-NN, foram selecionadas como alternativas as métricas 'minkowski','euclidean' e 'manhattan' conforme exemplo do kaggle;
* **n_jobs**: este parâmetro foi setado como -1 para representar que está usando todos os processadores.

In [9]:
params['KNN'] = {
    'n_neighbors': randint(3,10),
    'weights':['uniform', 'distance'],
    'algorithm':['auto', 'ball_tree','kd_tree','brute'],
    'metric' : ['minkowski','euclidean','manhattan'],
}

### LVQ

Para a definição do dicionário que foi utilizado na função RandomizedSearchCV foram levados em consideração os parâmetros de entrada do modelo que foi criado:

* **n_codebooks**: refere-se ao número de codebooks (vetores de referência) que serão utilizados pelo modelo. Quanto maior o número de codebooks, maior a capacidade do modelo de representar os dados, porém, isso também pode aumentar a complexidade do treinamento. Esse parâmetro define a quantidade de protótipos por classe que o algoritmo vai treinar e tentar ajustar durante o processo de aprendizado.

* **lrate**: é a taxa de aprendizado, ou seja, o quanto os codebooks serão atualizados a cada iteração de treinamento. Valores maiores de lrate farão com que os protótipos se ajustem mais rapidamente aos dados, mas também aumentam o risco de que o modelo se desajuste ou seja sensível ao ruído. Valores menores resultam em um ajuste mais gradual e controlado.

* **epochs**: define o número de épocas, ou seja, o número de vezes que o algoritmo passa por todo o conjunto de dados de treinamento. Mais épocas permitem que o modelo ajuste melhor os codebooks, mas também aumentam o tempo de treinamento. Se esse valor for muito alto, pode resultar em overfitting.

In [10]:
params['LVQ'] = {
    'n_codebooks': [5, 10],
    'lrate': [0.1, 0.01],
    'epochs': [20, 50, 100]
}

### AD

Para a definição do dicionário que foi utilizado na função RandomizedSearchCV foram levados em consideração as informações da função DecisionTreeClassifier()

* **criterion**: é um parâmetro que indica a função utilizada como critério de qualidade de uma separação e foi considerada para a busca duas das opções inclusas na biblioteca, sendo elas 'gini' e 'entropy';
* **splitter**: é um parâmetro que vai indicar qual estratégia será adotada para separação nos nós e foi considerada para a busca dos parâmetros 'best' e 'random';
* **max_depth**: esse parâmetro é um número inteiro que indica a profundidade máxima da árvore e foi considerado para este modelo qualquer número inteiro entre 3 e 10 escolhido de forma arbitrária. Vale ressaltar que árvores mais profundas podem gerar overfitting no treinamento;
* **min_samples_split**: esse parâmetro também é um número inteiro e indica o número mínimo de amostras necessárias para separar internamente um nó e seu valor default é 2. Para este modelo, foi considerado qualquer número inteiro entre 2 e 10 escolhido de forma arbitrária;
* **min_samples_leaf**: esse parâmetro é um valor inteiro que indica o número mínimo de amostras necessários em uma folha. Para este modelo, foi considerado qualquer número inteiro entre 2 e 5;
* **min_weight_fraction_leaf**: este parâmetro é um float que indica a fração mínima da soma de todos os pesos de um leaf node sendo o default 0.0. Para este modelo esse valor não foi alterado;
* **max_features**: este parâmetro indica o número de variáveis consideradas quando se busca a separação ideal. Para este modelo foi considerado 'auto', 'sqrt' e 'log2' como indicado na biblioteca;

In [11]:
params['DTR'] = {
    'criterion':['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': randint(3,10),
    'min_samples_split': randint(2,10),
    'min_samples_leaf': randint(2,5),
    'max_features': ['sqrt', 'log2'],
}

### SVM

Para a definição do dicionário que foi utilizado na função RandomizedSearchCV foram levados em considerações as informações da função SVC()

* **C**: este parâmetro é um float que tem como intuito regularizar o modelo e deve ser positivo. Foram definidos diversos valores possíveis para este modelo;
* **kernel**: com este parâmetro é possível indicar que tipo de kernel vai ser usado no modelo. Para este modelo o kernel pode ser do tipo 'rbf', 'linear', 'poly' ou 'sigmoid';
* **gamma**: este parâmetro é utilizado quando o kernel for do tipo rbf, linear ou sigmoid e estão sendo avaliados diversos valores inteiros.

In [12]:
params['SVC'] = {
    'C': [0.1,1,5,10,20,50,100],
    'kernel': ['linear','poly','rbf','sigmoid'],
    'gamma': [1,5,10,50,100,500],
}

#### MLP

Para a definição do dicionário que foi utilizado na função RandomizedSearchCV foram levados em considerações as informações da função MLPClassifier()

* **hidden_layer_sizes**: este parâmetro é uma tupla que indica a quantidade de neurônio em uma determinada camada e para este algoritmo foi considerado que os modelos podem utilizar as seguintes opções [(5, 5), (10, 10), (15, 15)];
* **activation**: este parâmetro indica a função de ativação utilizada pelo MLP e foram considerados 'identity', 'relu', 'logistic' para a criação dos modelos. A função de ativação 'tanh' não foi considerada pois em testes preliminares apresentou erros do tipo 'NaN';
* **solver**: este parâmetro se refere qual foi o solver selecionado para a otimização dos pesos do MLP e para a geração dos modelos foram considerados o default 'adam', 'lbfgs' e 'sgd';
* **alpha**: parâmetro float que indica a força do termo de regularização L2 e não foi alterado;
* **batch_size**: é um inteiro que indica a quantidade de subgrupos para os otimizadores estocásticos e foi considerado o default 'auto';
* **learning_rate**: indica a taxa de aprendizado dos pesos utilizados pelo MLP e foi utilizado o padrão constante;
* **learning_rate_init**: é um float que indica a taxa de aprendizado inicial considerada e para a geração dos modelos foi considerado que a taxa inicial pode assumir os seguintes valores [0.1, 0.01, 0.001, 0.0001];
* **power_t**: é uma potência para a escala inversa da taxa de aprendizado e foi utilizado o padrão 0.5;
* **max_iter**: é um parâmetro inteiro que indica o número máximo de iterações e para este modelo foram considerados os seguintes valores[50, 100, 150];
* **random_state**: o parâmetro random_state foi setado em 0 para controlar a aleatoriedade do estimador;
* **tol**: indica a tolerância de otimização e foi considerado o default 1e-4.

In [13]:
params['MLP'] = {
    'hidden_layer_sizes': [(5, 5), (10, 10), (15, 15)],
    'activation': ['identity', 'relu', 'logistic'],
    'solver': ['adam', 'lbfgs', 'sgd'],
    'learning_rate_init': [0.1, 0.01, 0.001, 0.0001],
    'max_iter': [50, 100, 150],
    'random_state': [0]
}

## Execução da Busca Randomizada

In [16]:
# Inicializando dicionário de resultados
resultados = {}

# Gerando K-Folds
cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

In [17]:
# Realizando a busca randomizada para cada um dos 4 modelos em questão
for name in models.keys():

    search_rf = RandomizedSearchCV(
        models[name],
        params[name],
        n_iter=5, 
        n_jobs=-1, # Usando todos os processadores
        cv=cv, # Aplicando os splits
        random_state=0, # Definindo Random State para melhor reprodutibilidade
        verbose=1 # Aumentando a verbose para ter uma melhor noção do andamento da busca
    )

    resultados[name] = search_rf.fit(X.values, Y.values)

    results = f'Modelo {name} obteve melhores parâmetros {resultados[name].best_params_} e melhor score {resultados[name].best_score_} \n'
    
    print(results)
    
    with open('C:/Users/55819/Downloads/randomized_cv_best_params.txt', 'a') as file:
        file.write(results)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Modelo DTR obteve melhores parâmetros {'criterion': 'gini', 'max_depth': 7, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 2, 'splitter': 'best'} e melhor score 0.5390187718363239 

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Modelo KNN obteve melhores parâmetros {'algorithm': 'brute', 'metric': 'euclidean', 'n_neighbors': 6, 'weights': 'distance'} e melhor score 0.6513771540238052 

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Modelo SVC obteve melhores parâmetros {'kernel': 'linear', 'gamma': 10, 'C': 5} e melhor score 0.5333233848522532 

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Modelo LVQ obteve melhores parâmetros {'n_codebooks': 10, 'lrate': 0.01, 'epochs': 100} e melhor score 0.468841830994256 

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Modelo MLP obteve melhores parâmetros {'solver': 'lbfgs', 'random_state': 0, 'max_iter': 150, 'le

c:\Users\55819\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## Validação Cruzada

Selecionando melhores parâmetros obtidos a partir da busca randomizada e inicializando novos modelos com as melhores configurações. Esses modelos, em seguida, estão passando pelo procedimento da validação cruzada com 10 folds (requisito mínimo para execução do teste de Friedman com qualidade).

In [18]:
best_models = {}

In [19]:
# KNN
knn_params = resultados['KNN'].best_params_
best_models['KNN'] = KNeighborsClassifier(
                n_neighbors=knn_params['n_neighbors'],
                weights=knn_params['weights'],
                algorithm=knn_params['algorithm'],
                metric=knn_params['metric']
                )

In [20]:
# LVQ
lvq_params = resultados['LVQ'].best_params_
best_models['LVQ'] = LVQ(
                n_codebooks = lvq_params['n_codebooks'],
                lrate = lvq_params['lrate'],
                epochs = lvq_params['epochs'],
                )

In [21]:
# DTR
dtr_params = resultados['DTR'].best_params_
best_models['DTR'] = DecisionTreeClassifier(
                criterion = dtr_params['criterion'],
                splitter = dtr_params['splitter'],
                max_depth = dtr_params['max_depth'],
                min_samples_split = dtr_params['min_samples_split'],
                min_samples_leaf = dtr_params['min_samples_leaf'],
                max_features = dtr_params['max_features'],
                random_state = 0
                )

In [22]:
# SVC
svc_params = resultados['SVC'].best_params_
best_models['SVC'] = SVC(
                C = svc_params['C'],
                kernel = svc_params['kernel'],
                gamma = svc_params['gamma'],
                probability=True
                )


In [23]:
# MLP
mlp_params = resultados['MLP'].best_params_
best_models['MLP'] = MLPClassifier(
                hidden_layer_sizes = mlp_params['hidden_layer_sizes'],
                activation = mlp_params['activation'],
                solver = mlp_params['solver'],
                learning_rate_init = mlp_params['learning_rate_init'],
                max_iter = mlp_params['max_iter'],
                random_state = 0
                )
                

In [24]:
models_to_cv = []

In [25]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, auc, accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import StratifiedKFold

fitted_models = {}
all_metrics = {}
aucs = {}
for name in models.keys():
    cv = StratifiedKFold(n_splits=10, shuffle=False)

    metrics_per_split = {
        'train_accuracy': [],
        'test_accuracy': [],
        'f1_score': [],
        'precision': [],
        'recall': [],
    }
    all_auc = []
    fold = 0
    for train_idx, test_idx in cv.split(X.values, Y.values):
        print(f'Fold: {fold}')

        X_train_split, y_train_split = X.values[train_idx], Y.values[train_idx]
        X_test_split, y_test_split = X.values[test_idx], Y.values[test_idx]

        # Treinando o modelo
        fitted_models[name] = best_models[name].fit(X_train_split, y_train_split)

        # Predições nos conjuntos de treinamento e teste
        y_train_pred_split = fitted_models[name].predict(X_train_split)
        y_test_pred_split = fitted_models[name].predict(X_test_split)
        y_score = fitted_models[name].predict_proba(X_test_split)

        # Binariza os rótulos de saída
        y_bin = label_binarize(y_test_split, classes=fitted_models[name].classes_)
        n_classes = y_bin.shape[1]

        # Cálculo das métricas
        train_accuracy = accuracy_score(y_train_split, y_train_pred_split)
        test_accuracy = accuracy_score(y_test_split, y_test_pred_split)
        f1 = f1_score(y_test_split, y_test_pred_split, average='weighted')
        prec = precision_score(y_test_split, y_test_pred_split, average='weighted')
        rec = recall_score(y_test_split, y_test_pred_split, average='weighted')

        # Calcular a curva ROC e a AUC para cada classe
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for idx in range(n_classes):
            fpr[idx], tpr[idx], _ = roc_curve(y_bin[:, idx], y_score[:, idx])
            roc_auc[fitted_models[name].classes_[idx]] = auc(fpr[idx], tpr[idx])

        # Salvando as métricas
        metrics_per_split['train_accuracy'].append(train_accuracy)
        metrics_per_split['test_accuracy'].append(test_accuracy)
        metrics_per_split['f1_score'].append(f1)
        metrics_per_split['precision'].append(prec)
        metrics_per_split['recall'].append(rec)
        all_auc.append(roc_auc)

        fold += 1

    aucs[name] = pd.DataFrame(all_auc).mean(axis=0).to_dict()
    all_metrics[name] = metrics_per_split
    metrics_df = pd.DataFrame(metrics_per_split)

    # Salvando métricas obtidas em um Excel
    metrics_df.to_excel(f'C:/Users/55819/Downloads/metrics_cv_wine_{name}_com_auc.xlsx', index=False)


c:\Users\55819\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold: 0


c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py:1133: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py:1133: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\55819

Fold: 1
Fold: 2
Fold: 3
Fold: 4
Fold: 5
Fold: 6
Fold: 7


c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py:1133: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_divi

...

c:\Users\55819\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py:1133: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


Fold: 8


c:\Users\55819\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py:1133: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


Fold: 9


c:\Users\55819\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\55819\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py:1133: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


In [28]:
pd.DataFrame(aucs)

,DTR,KNN,SVC,LVQ,MLP
3,0.485308,0.576583,0.699194,0.500000,0.799722
4,0.613878,0.665223,0.759780,0.536288,0.805125
5,0.714333,0.685716,0.773081,0.684566,0.755443
6,0.552124,0.548545,0.581229,0.547727,0.581606
7,0.741531,0.732548,0.764165,0.629005,0.797840
8,0.735069,0.634893,0.733669,0.535793,0.839079
9,0.710632,0.496764,0.930971,0.500000,0.891217
